In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered the seventh avatar (incarnation) of the god Vishnu. He is the main character in the ancient Indian epic, the Ramayana, which is one of the most revered and influential texts in Hinduism.

According to Hindu mythology, Lord Rama was born as a prince in the kingdom of Ayodhya, to King Dasharatha and Queen Kausalya. He is known for his exceptional virtues, such as truthfulness, courage, and devotion to his family and duty.

The story of Lord Rama is as follows:

* He was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Rama defeated Ravana and rescued Sita, returning t

In [9]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers

In [17]:

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........

Human: i'm tired
Chatbot: I'm so sorry to hear that you're feeling tired. It can be really overwhelming and exhausting, both physically and emotionally. Would you like to talk about what's been going on and how you've been feeling lately? Sometimes sharing your thoughts and feelings can help you feel a bit better and gain some perspective.

It's also important to remember that feeling tired can be a symptom of many things, including stress, anxiety, or depression. If you're feeling like you're struggling to cope with your emotions or daily life, it might be helpful to talk to someone who can support you, like a mental health professional.

Remember, you don't have to go through this alone. I'm here to listen and offer support in any way I can. What's been on your mind lately, and how can I help you feel more supported and less tired?

Human: exit
Chatbot: Take Care of yourself, Goodbye!


In [13]:
!pip install gradio

In [22]:
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

import os
import gradio as gr

def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vector_db = Chroma.from_documents(
        texts, embeddings, persist_directory="./chroma_db"
    )
    vector_db.persist()
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt = PromptTemplate(
        template="""
You are a compassionate mental health chatbot.
Use the context below to answer the question.

Context:
{context}

Question:
{question}

Answer:
""",
        input_variables=["context", "question"]
    )

    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

llm = initialize_llm()

db_path = "./chroma_db"
embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    vector_db = Chroma(
        persist_directory=db_path,
        embedding_function=embeddings
    )

qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(message, history):
    return qa_chain.run(message)

app = gr.ChatInterface(
    fn=chatbot_response,    title="🧠 Mental Health Chatbot"
)

app.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3707308951c1765446.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
